In [1]:
# Code<1> Get library & Data
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

# 데이터 전처리 방식 지정
transform = transforms.Compose([
    transforms.ToTensor(), # Data to pytorch tensor format
    transforms.Normalize(mean=(0.5, ), std=(0.5, ))
])

# Get Mnist Datasets
mnist = datasets.MNIST(root='data', download=True, transform=transform)

# Get data sizeof batch_size
dataloader = DataLoader(mnist, batch_size=60, shuffle=True)

In [12]:
# GAN의 2가지 요소 중 하나인 생성자(Generator) 생성
# 생성자는 랜덤 벡터 'z'를 입력받아 가짜 이미지를 출력하는 함수
# 생성자는 단순한 분포를 사람 얼굴 이미지와 같은 복잡한 분포로 매핑하는 함수
# 생성자 모델에 충분한 수의 매개 변수가 있다면 어떤 복잡한 분포도 근사할 수 있다는 것이 알려져 있다. 
# 'z'벡터가 존재하는 공간을 잠재공간이라고 부른다. 
# 잠재 공간의 크기에는 제한이 없으나 나타내려고 하는 대상의 정보를 충분히 담을 수 있을 만큼 커야 한다. 
# GAN은 우리가 이해할 수는 없는 방식이지만 'z'벡터의 값을 이미지의 속성에 매핑시킨다. 

In [13]:
# GAN Generator
# input Random vector 'z' output fake image
class Generator(nn.Module):
    # Network Structure
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=100, out_features=256),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Linear(in_features=256, out_features=512),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Linear(in_features=512, out_features=1024),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Linear(in_features=1024, out_features=28*28),
            nn.Tanh()
        )
    
    # batch_size * 100 크기의 랜덤 벡터를 받아
    # 이미지를 batch_size * 1 * 28 * 28 크기로 출력한다.
    def forward(self, inputs):
        return self.main(inputs).view(-1, 1, 28, 28)

In [14]:
# 4개의 선형 레이어 이용 (Linear Layer, Fully Connected layer, Linear Transformation)
# 선형 레이어는 속해있는 모든 뉴런이 이전 레이어의 모든 뉴런과 연결되는 가장 단순한 구조의 레이어
# 활성함수로는 LeakyReLU
# : 각 뉴런의 출력값이 0보다 높으면 그대로 놔두고, 0보다 작으면 정해진 작은 숫자를 곱하는 간단한 함수
# 마지막 레이어에서는 출력값을 픽셀값의 범위로 (-1~1)만들어주기위해 Tanh사용

In [15]:
# GAN Discriminator
# input image ouput real or fake
class Discriminator(nn.Module):
    # Network Structure
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=1024),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Dropout(inplace=False),
            nn.Linear(in_features=1024, out_features=512),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Dropout(inplace=False),
            nn.Linear(in_features=512, out_features=256),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Dropout(inplace=False),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid()
        )
    
    # batch_size * 1 * 28 * 28 크기의 이미지를 받아
    # 이미지가 진짜일 확률을 0~1 사이로 출력한다.
    def forward(self, inputs):
        inputs = inputs.view(-1, 28*28)
        return self.main(inputs)

In [16]:
# Generate Objects of Generator, Discriminator
G = Generator()
D = Discriminator()

In [17]:
# 구분자는 이미지를 입력으로 받고 그 이미지가 진짜일 확률을 0~1 사이의 숫자 하나로 출력하는 함수
# 입력값으로 이미지 크기인 28*28개의 변수를 받은 뒤 마지막에는 확률값을 나타내는 숫자 하나를 출력한다.

# 레이어마다 들어간 드롭아웃(Dropout) 학습 시에 무작위로 절반의 뉴런을 사용하지 않도록 한다.
# 이를 통해 모델이 과적합(Overfitting)되는 것을 방지할 수 있고, 구분자가 생성자보다 지나치게 빨리 학습되는 것도 막을 수 있다.
# 마지막 레이어에서는 출력값을 0~1 사이로 만들기 위해서 활성 함수로 Sigmoid를 넣었다.

In [18]:
# 손실 함수와 최적화 기법 지정하기

# Binary Cross Entropy Loss
criterion = nn.BCELoss()

G_optimizer = Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
D_optimizer = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [19]:
# 학습 결과 시각화하기
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

def square_plot(data, path):
    """Take an array of shape (n, height, width) or (n, height, width , 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""

    if type(data) == list:
	    data = np.concatenate(data)
    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())

    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))

    padding = (((0, n ** 2 - data.shape[0]) ,
                (0, 1), (0, 1))  # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data , padding, mode='constant' , constant_values=1)  # pad with ones (white)

    # tilethe filters into an image
    data = data.reshape((n , n) + data.shape[1:]).transpose((0 , 2 , 1 , 3) + tuple(range(4 , data.ndim + 1)))

    data = data.reshape((n * data.shape[1] , n * data.shape[3]) + data.shape[4:])

    plt.imsave(path, data, cmap='gray')

In [20]:
# 학습하기 위해서는 모델을 평가할 수 있어야 한다.
# 구분자의 출력값은 이미지가 진짜일 확률
# 이 확률이 정답과 얼마나 가까운지를 측정하기 위해서 Binary Cross Entropy Loss Function을 사용한다. 
# 이 함수는 구분자가 출력한 확률값이 정답에 가까우면 낮아지고, 멀어지면 높아진다.

In [ ]:
### 모델 학습을 위한 반복문
# 데이터셋을 100번 돌며 학습한다.
for epoch in range(100):
    
    # 한번에 batch_size만큼 데이터를 가져온다.
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
        
        # 데이터를 pytorch의 변수로 변환한다.
        real_data = Variable(real_data)

        ### 구분자 학습시키기

        # 이미지가 진짜일 때 정답 값은 1이고 가짜일 때는 0이다.
        # 정답지에 해당하는 변수를 만든다.
        target_real = Variable(torch.ones(batch_size, 1))
        target_fake = Variable(torch.zeros(batch_size, 1))
            
        # 진짜 이미지를 구분자에 넣는다.
        D_result_from_real = D(real_data)
        # 구분자의 출력값이 정답지인 1에서 멀수록 loss가 높아진다.
        D_loss_real = criterion(D_result_from_real, target_real)

        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
        
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 구분자의 출력값이 정답지인 0에서 멀수록 loss가 높아진다.
        D_loss_fake = criterion(D_result_from_fake, target_fake)
        
        # 구분자의 loss는 두 문제에서 계산된 loss의 합이다.
        D_loss = D_loss_real + D_loss_fake
        
        # 구분자의 매개 변수의 미분값을 0으로 초기화한다.
        D.zero_grad()
        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        D_loss.backward()
        # 최적화 기법을 이용해 구분자의 매개 변수를 업데이트한다.
        D_optimizer.step()
        

        # train generator G

        ### 생성자 학습시키기
        
        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        
        
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 생성자의 입장에서 구분자의 출력값이 1에서 멀수록 loss가 높아진다.
        G_loss = criterion(D_result_from_fake, target_real)
        
        # 생성자의 매개 변수의 미분값을 0으로 초기화한다.
        G.zero_grad()
        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        G_loss.backward()
        # 최적화 기법을 이용해 생성자의 매개 변수를 업데이트한다.
        G_optimizer.step()


torch.save(G.state_dict(), "gan_generator.pkl")
torch.save(D.state_dict(), "gan_discriminator.pkl")
with open('gan_train_history.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

generated_image_array = [imageio.imread(generated_image) for generated_image in generated_images]
imageio.mimsave(result_dir + '/GAN_generation.gif', generated_image_array, fps=5)